In [11]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [2]:
class Testing_Dataset(Dataset):
    def __init__(self,csv_file,root_dir,transform):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        features = self.data.iloc[idx]
        features = features.drop(labels='id',axis=0)
        
        
        sample = {"features":features}
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample

In [3]:
class to_tensor(object):
    def __call__(self,sample):
        features = sample['features']
        features = np.array([features])
        features = features.astype('float32')
        features = torch.from_numpy(features)
        features = features.float()
        
        return {"features":features}
        

In [4]:
transformed_dataset_testing = Testing_Dataset(csv_file="test.csv",
                                       root_dir="/Users/abdulsalamyazid/Desktop/Projects/Kaggle",
                                       transform = to_tensor())


In [8]:
for i in range(len(transformed_dataset_testing)):
    sample = transformed_dataset_testing[i]

    print(i, sample['features'].size(0))

    if i == 3:
        break

0 1
1 1
2 1
3 1


In [9]:
testloader = torch.utils.data.DataLoader(transformed_dataset_testing,batch_size=4, shuffle=True,num_workers=2)


In [12]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(93,128)
        self.fc2 = nn.Linear(128,256)
        self.fc3 = nn.Linear(256,512)
        self.fc4 = nn.Linear(512,64)
        self.fc5 = nn.Linear(64,9)
        
    def forward(self,x):
        #drouput_activation1 = nn.Dropout(p=0.3)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

In [15]:
net = Net()
checkpoint = torch.load("OttoGroup.pth")
net.load_state_dict(checkpoint["model_state_dict"])

<All keys matched successfully>

In [ ]:

with torch.no_grad():
    for i , data in enumerate(testloader):
        features = data['features']
        outputs = net(features)
        _, predictions = torch.max(outputs,1)